<a href="https://colab.research.google.com/github/gtrofimiuk/ML_course/blob/HW1/MusicGenres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Датасет https://www.kaggle.com/datasets/vicsuperman/prediction-of-music-genre/data

Изначально я скачивал через код ниже, но, оказалось, что DATA_SOURCE_MAPPING временный. В принципе, датасет уже есть в репозитории, ну или можно взять из kaggle

В любом случае, папки создам

In [3]:
import os
os.makedirs("sample_data/prediction-of-music-genre", 0o777, exist_ok=True)

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'prediction-of-music-genre:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1690118%2F2769452%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240131%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240131T163614Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da7baa0c32bc50dbc5f8a6acb57083e776b5012d59bae1d3c5933a39ffc813ef54f87ae16c8cdf17e0b7283cf4db66ddbdc8bbdf868d445fb511d710633eedb4fc62278a68f8b3c8eb85b128f44e128f069181ac5c585c014607f2fe9ee77cba72572d05edf6ff87243b28de02b6f261e905d1dd48c088c9a7463cddd55b5005bc5954343682d3511e6393190d67993e328dc027c8a82f1bf3e47cc52239ac5de04488d4bac6a47de135b7a3645af62d0c07639f1b32b0ac039a5cf9d82805c4e6c22f98d967f9cb5aa6d26e2f2af56f63a1301058fe6e540057e95a0ce6c17e030d47265e5f5e073b66760f188375e21ebcbd0707ae64da63257d1c13d5e4a00'


for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join("sample_data", directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [5]:
!pip install basemap > None

In [6]:
!pip install --upgrade matplotlib > None # need refresh enviroment after installation

In [9]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.cbook import boxplot_stats
from mpl_toolkits.basemap import Basemap

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA

%matplotlib inline

# Данные

Набор данных состоит из различных параметров музыкальных треков

In [12]:
dataset_path = "sample_data/prediction-of-music-genre/music_genre.csv"
data = pd.read_csv(dataset_path)

In [15]:
data.head()

,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
0,32894.0,Röyksopp,Röyksopp's Night Out,27.0,0.00468,0.652,-1.0,0.941,0.79200,A#,0.115,-5.201,Minor,0.0748,100.889,4-Apr,0.759,Electronic
1,46652.0,Thievery Corporation,The Shining Path,31.0,0.01270,0.622,218293.0,0.890,0.95000,D,0.124,-7.043,Minor,0.0300,115.00200000000001,4-Apr,0.531,Electronic
2,30097.0,Dillon Francis,Hurricane,28.0,0.00306,0.620,215613.0,0.755,0.01180,G#,0.534,-4.617,Major,0.0345,127.994,4-Apr,0.333,Electronic
3,62177.0,Dubloadz,Nitro,34.0,0.02540,0.774,166875.0,0.700,0.00253,C#,0.157,-4.498,Major,0.2390,128.014,4-Apr,0.270,Electronic
4,24907.0,What So Not,Divide & Conquer,32.0,0.00465,0.638,222369.0,0.587,0.90900,F#,0.157,-6.266,Major,0.0413,145.036,4-Apr,0.323,Electronic


In [17]:
data.shape

(50005, 18)

## Признаки в датасете

In [18]:
data.dtypes

instance_id         float64
artist_name          object
track_name           object
popularity          float64
acousticness        float64
danceability        float64
duration_ms         float64
energy              float64
instrumentalness    float64
key                  object
liveness            float64
loudness            float64
mode                 object
speechiness         float64
tempo                object
obtained_date        object
valence             float64
music_genre          object
dtype: object

Как можно увидеть, изначально у нас есть 18 описание которых можно найти на https://developer.spotify.com/documentation/web-api/reference/get-audio-features

(Отмечу, что на данный момент Spotify запрещает использовать свое API для тренировки моделей машинного обучения, но, в РФ они не работают, да и датасет лежит в открытом доступе, так что ладно. Но вот обновить его уже сложно)

* instance_id - Идентификатор, без особого смысла
* artist_name - Исполнитель песни
* track_name - Название трека

**target: music_genre**


## Идеи по конструированию признаков

Я все же вижу смысл модели посмотреть на элементарные признаки из текста.

Количество слов у исполнителя,
Наличие скобок в тексте песни


На будущее

Преобразовать тональности по кварто-квинтовому кругу в круг
И по хроматической гамме